In [1]:
import tensorflow as tf
import plotly.express as px

In [2]:
directory = 'images/final/'
width = 128
height = 128
channels = 3
image_size = (width, height)
input_shape = (width, height, channels)
batch_size = 32

In [3]:
def plot_images(images, labels):
    fig = px.imshow(images, width=width, height=height)
    fig.update_layout(
        title=f"Label: {labels}",
        width=300,
        height=300,
    )
    fig.show()

In [68]:
def plot_n_images(ds, n):
    for image, label in ds.take(1):
        for i in range(n):
            print(image[i][0][0])
            print(image[i].shape)
            plot_images(image[i], class_names[int(tf.argmax(tf.reshape(label[i], [-1, 1]), axis=0))])

In [87]:
train_ds, val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    batch_size=batch_size,
    validation_split=0.2,
    subset="both",
    labels="inferred",
    label_mode="categorical",
    image_size=image_size,
    crop_to_aspect_ratio=True,
    interpolation="bilinear",
    color_mode="rgba",
    shuffle=True,
    seed=905,
)

class_names = train_ds.class_names

Found 13894 files belonging to 905 classes.
Using 11116 files for training.
Using 2778 files for validation.


In [88]:
plot_n_images(train_ds, 1)

tf.Tensor([0. 0. 0. 0.], shape=(4,), dtype=float32)
(128, 128, 4)


In [89]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input

In [90]:
flip_and_rotate = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1, fill_mode="constant"),
    # tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
])

def preprocess_image(image, label):
    # Convert the image to RGB (if it's not already)
    # image = tf.cond(tf.equal(tf.shape(image)[-1], 4),
    #                 lambda: image[..., :3],
    #                 lambda: image)
    
    # # Convert the image to float32
    # image = tf.cast(image, tf.float32)

    # # Check if the image has an alpha channel
    # if tf.shape(image)[-1] == 4:
    #     # Extract the alpha channel
    #     alpha = image[..., 3]

    #     # Create a mask of the alpha channel
    #     mask = tf.where(alpha > 0, 1.0, 0.0)

    #     # Replace the alpha channel with a solid white background
    #     bg = tf.ones_like(alpha, dtype=tf.float32)
    #     image = tf.concat([image[..., :3], tf.expand_dims(bg, axis=-1)], axis=-1)

    #     # Apply the mask to the image
    #     mask = tf.expand_dims(mask, axis=-1)
    #     image = image * mask
    
    # Normalize the pixel values to be between 0 and 1
    image = preprocess_input(image)
    
    return image, label

def prepare(ds, shuffle=False, augment=False):
    ds = ds.map(lambda x, y: (preprocess_image(x, y)), num_parallel_calls=tf.data.AUTOTUNE)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    
    # Use buffered prefecting on all datasets
    if shuffle:
        ds = ds.shuffle(1000)

    # ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    return ds

In [91]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)

In [92]:
plot_n_images(train_ds, 5)

tf.Tensor([6.55896   7.259041  6.4812927 0.7640209], shape=(4,), dtype=float32)
(128, 128, 4)


tf.Tensor([2.6551857 1.4794025 1.6481838 3.3624344], shape=(4,), dtype=float32)
(128, 128, 4)


tf.Tensor([5.267212  4.725281  3.4792786 1.4317112], shape=(4,), dtype=float32)
(128, 128, 4)


tf.Tensor([0. 0. 0. 0.], shape=(4,), dtype=float32)
(128, 128, 4)


tf.Tensor([8.937027  7.6552353 7.1669617 6.5562973], shape=(4,), dtype=float32)
(128, 128, 4)


In [8]:
# Define the EfficientNetB4 model
base_model = EfficientNetB4(include_top=False, input_shape=input_shape, pooling='avg', weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Define the model inputs
inputs = tf.keras.Input(shape=input_shape, batch_size=batch_size)

# Preprocess the inputs
x = base_model(inputs, training=False)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs, x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['top_k_categorical_accuracy'])

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(32, 128, 128, 3)]       0         
                                                                 
 efficientnetb4 (Functional)  (None, 1792)             17673823  
                                                                 
 dense (Dense)               (32, 512)                 918016    
                                                                 
 dropout (Dropout)           (32, 512)                 0         
                                                                 
 dense_1 (Dense)             (32, 256)                 131328    
                                                                 
 dropout_1 (Dropout)         (32, 256)                 0         
                                                                 
 dense_2 (Dense)             (32, 905)                 232585

In [10]:
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\valen